In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pickle

In [6]:
from sqlalchemy import create_engine
import cx_Oracle
lib_dir = r"C:\Users\jakub\Desktop\instantclient_21_13"
cx_Oracle.init_oracle_client(lib_dir=lib_dir)


# niezbedny downgrade sqlalchemy
#!pip install --force-reinstall sqlalchemy==1.4.40
#!pip install --force-reinstall pymysql==1.0.2

lib_dir = r"C:\Users\jakub\Desktop\instantclient_21_13"
#cx_Oracle.init_oracle_client(lib_dir=lib_dir)
#print(cx_Oracle.__version__)

oracle_pass = 's101374'
oracle_engine = create_engine("oracle+cx_oracle://s101374:"+oracle_pass+"@"+"217.173.198.135:1521/?service_name=tpdb&encoding=UTF-8")

In [7]:
#!pip install redis
import redis


#r = redis.Redis(host='127.0.0.1', port=6379, db=0)
r = redis.Redis(host='localhost', port=6379)

In [10]:
#r.get_connection_kwargs()

In [9]:
#!pip install --force-reinstall sqlalchemy==1.4

In [8]:
#uslugi obslugiwane przez danego pracownika

def read_from_database1(imie, nazwisko):
    querry=f"select * from usluga where id_pracownik=(select id_pracownik from pracownik where imie='{imie}' and nazwisko='{nazwisko}')"
    #print(querry)
    record = oracle_engine.execute(querry).fetchone()
    return record

#%time 
#record1 = read_from_database1("Aleksy", "Czerwiński")
#record1

In [9]:
CACHE_DURATION_SEC = 3600

def read_from_cache1(imie, nazwisko):
    data = r.get(imie+":"+nazwisko)
    if data:
        data = pickle.loads(data)
    return data

def write_to_cache1(imie, nazwisko, data):
    data = pickle.dumps(data)
    r.set(imie+":"+nazwisko, data, ex=CACHE_DURATION_SEC)

In [10]:
# uslugi powiazane z dana firma
def read_from_database2(nazwa_firmy):
    querry=f"select * from usluga where id_klient=(select id_klient from klient where id_firma=(select id_firma from firma where nazwa_firmy='{nazwa_firmy}')FETCH FIRST ROW ONLY)"
    #print(querry)
    record = oracle_engine.execute(querry).fetchone()
    return record

#%time 
#record2 = read_from_database2("Elektomaks")
#record2

In [11]:
def read_from_cache2(nazwa_firmy):
    data = r.get(nazwa_firmy)
    if data:
        data = pickle.loads(data)
    return data

def write_to_cache2(nazwa_firmy, data):
    data = pickle.dumps(data)
    r.set(nazwa_firmy, data, ex=CACHE_DURATION_SEC)

In [12]:
#cena podzespolu o podanych informacjach

def read_from_database3(nazwa_podzespolu):
    querry=f"select cena_brutto_podzespolu from egzemplarz where id_podzespol=(select id_podzespol from podzespol where nazwa_podzespolu='{nazwa_podzespolu}'FETCH FIRST ROW ONLY)"
    #print(querry)
    record = oracle_engine.execute(querry).fetchone()
    return record

#%time 
record3 = read_from_database3("Core i7")
#print(record3)


In [13]:
def read_from_cache3(nazwa_podzespolu):
    data = r.get(nazwa_podzespolu)
    if data:
        data = pickle.loads(data)
    return data

def write_to_cache3(nazwa_podzespolu, data):
    data = pickle.dumps(data)
    r.set(nazwa_podzespolu, data, ex=CACHE_DURATION_SEC)

In [ ]:
#AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [15]:
def get_record1(imie, nazwisko):

    info = "CACHE"
    record = read_from_cache1(imie, nazwisko)
    
    if not record:
        info = "SQL DATABASE"
        record = read_from_database1(imie, nazwisko)
        write_to_cache1(imie, nazwisko, record)    
        
    return (info, record)

%time print(get_record1("Aleksy", "Czerwiński"))

('SQL DATABASE', (6, '1', '0', 'komputer04', None, 'POMOCYYYY', datetime.datetime(2020, 4, 4, 0, 0), None, Decimal('440'), 3, 15))
CPU times: total: 0 ns
Wall time: 445 ms


In [16]:
def get_record2(nazwa_firmy):

    info = "CACHE"
    record = read_from_cache2(nazwa_firmy)
    
    if not record:
        info = "SQL DATABASE"
        record = read_from_database2(nazwa_firmy)
        write_to_cache2(nazwa_firmy, record)    
        
    return (info, record)

%time print(get_record2("Elektomaks"))

('SQL DATABASE', (3, '1', '0', 'Laptop Intel', None, 'Przestalo dzialac... tak, samo', datetime.datetime(2005, 9, 3, 0, 0), None, Decimal('131'), 7, 11))
CPU times: total: 0 ns
Wall time: 472 ms


In [17]:
def get_record3(nazwa_podzespolu):

    info = "CACHE"
    record = read_from_cache3(nazwa_podzespolu)
    
    if not record:
        info = "SQL DATABASE"
        record = read_from_database3(nazwa_podzespolu)
        write_to_cache3(nazwa_podzespolu, record)    
        
    return (info, record)

%time print(get_record3("Core i7"))

('SQL DATABASE', (Decimal('998.47'),))
CPU times: total: 15.6 ms
Wall time: 338 ms


In [18]:
print("DRUGI RAZ TO SAMO - TYM RAZEM Z REDIS")

%time print(get_record1("Aleksy", "Czerwiński"))
%time print(get_record2("Elektomaks"))
%time print(get_record3("Core i7"))

DRUGI RAZ TO SAMO - TYM RAZEM Z REDIS
('CACHE', (6, '1', '0', 'komputer04', None, 'POMOCYYYY', datetime.datetime(2020, 4, 4, 0, 0), None, Decimal('440'), 3, 15))
CPU times: total: 0 ns
Wall time: 3.02 ms
('CACHE', (3, '1', '0', 'Laptop Intel', None, 'Przestalo dzialac... tak, samo', datetime.datetime(2005, 9, 3, 0, 0), None, Decimal('131'), 7, 11))
CPU times: total: 0 ns
Wall time: 2.15 ms
('CACHE', (Decimal('998.47'),))
CPU times: total: 0 ns
Wall time: 2.06 ms


In [16]:
# Dane do polaczenia z paza danych Oracle
#con = cx_Oracle.connect('s101374/'+'s101374'+'@217.173.198.135:1521/tpdb')